In [1]:
import pandas as pd
import json
from web3 import Web3
from datetime import datetime, timedelta
import time
import sys
import json
from collections import Counter

# Map token names to contract addresses
token_address_map = {
    'rETH': '0xae78736Cd615f374D3085123A210448E74Fc6393'
    # fill with rest
}

# Map market name to "Pool" contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './AAVE_Pool_ABI.json')
    # fill with rest
}

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/7e4f5238262543919688a59d0fef4a1d'))

In [31]:
# Find block number closest to timestamp with binary search
def find_block_by_timestamp(target_timestamp):
    low, high = 0, w3.eth.block_number
    while low < high:
        mid = (low + high) // 2
        mid_block_timestamp = w3.eth.get_block(mid).timestamp
        if mid_block_timestamp < target_timestamp:
            low = mid+1
        else:
            high = mid
    return low


# Helper to convert block number to readable timestamp string
def get_block_timestamp(block_num):
    block = w3.eth.get_block(block_num)
    timestamp = block.timestamp
    
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')



# Lists transactions of supplying rETH to specified markets
def supply_transactions(markets = ['AAVE'], timeframe = 30, token = 'rETH'):
    """
    Returns all "Supply" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: supply_transactions(AAVE, 90, rETH) returns all supply transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique supply transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    # List for supply_lists of all markets
    all_market_supply_list = []
    
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Start processing supply API calls for each market
    for market in markets:
        # Make sure the requested market is supported
        if market in contract_address_abi_map.keys():
            # Set up contract address and ABI for each market
            contract_address, abi_filepath = contract_address_abi_map[market]
    
            # Web3 setup
            with open(abi_filepath) as f:
                abi = json.load(f)

            smart_contract = w3.eth.contract(address=contract_address, abi=abi)
            
            # Here we will need some sort of if/else or switching logic to process unique calls for each market
            # In the meantime, we simply check for the only supported market, 'AAVE'
            if market == 'AAVE':
                supplies = smart_contract.events.Supply().get_logs(fromBlock=st_block,
                                                           toBlock='latest',
                                                           argument_filters={'reserve': token_address})
                # Consider simplifying supply_list.. don't know what information future markets will return, 
                # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
                supply_list = [
                    {
                        'Reserve': event['args']['reserve'],
                        'User (Wallet Address)': event['args']['user'],
                        'Amount (Wei)': event['args']['amount'],
                        'Amount (rETH)': event['args']['amount']/10**18,
                        'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                        'LogIndex': event['logIndex'],
                        'TransactionIndex': event['transactionIndex'],
                        'TransactionHash': event['transactionHash'].hex(),
                        'BlockHash': event['blockHash'].hex(),
                        'BlockNumber': event['blockNumber']
                    }
                    for event in supplies
                ]
                # Append each markets supply list results to a single all markets list
                all_market_supply_list.append(supply_list)
        # The market wasn't in our dictionary of supported markets
        else:
            print("UNSUPPORTED MARKET ENTERED")
            
    # Push withdrawls into dataframe
    recent_supplies = pd.DataFrame(all_market_supply_list[0])
    #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
    return recent_supplies
        
                
                

                

            
            
            
            
    
    
    
    

In [32]:
recent_supply = supply_transactions()
recent_supply

,Reserve,User (Wallet Address),Amount (Wei),Amount (rETH),Timestamp,LogIndex,TransactionIndex,TransactionHash,BlockHash,BlockNumber
0,0xae78736Cd615f374D3085123A210448E74Fc6393,0x7a0C9037fB4E51dd10a921E1eA8AAee07862F644,50000000000000000000,50.000000,2024-01-22 08:02:47,128,89,0x1920696ad27de888751a45e9c8881f71946129834b71...,0xd4b2068e2a68ed5857de83dd7bbf133b97c5bd35b84b...,19060961
1,0xae78736Cd615f374D3085123A210448E74Fc6393,0x6F30C82BaE8523C2F78100eB135d4310Ac6EF59B,1160002430791684646,1.160002,2024-01-22 08:30:35,174,143,0xdbc0fe3a74bacbcc7f00247ba8f59c83f93b83d07e32...,0x20b04f456af4f5046ecb856b14b63f68f3194b1591fe...,19061099
2,0xae78736Cd615f374D3085123A210448E74Fc6393,0x8C944510A73Ff90Cd6CB74A336B9075AD6c8bB94,3649982883807805399,3.649983,2024-01-23 00:01:23,321,119,0xb6ff55a55b981d24d43cb55a303f4e45d950cec085cf...,0x96c5268e33509f0bf28d22913847c5d99a20d9c0ceff...,19065677
3,0xae78736Cd615f374D3085123A210448E74Fc6393,0xa4B6508F51bA37765846A12d4362205ACef278e1,13676260612665375080,13.676261,2024-01-23 02:02:47,777,103,0x858a345943ea0da370e899f59669578a4657b90112c3...,0x0300c994c394ef78c07a828b6a9a727bee1538fa599a...,19066280
4,0xae78736Cd615f374D3085123A210448E74Fc6393,0xa4B6508F51bA37765846A12d4362205ACef278e1,17319092495185624533,17.319092,2024-01-23 02:10:23,236,102,0xcf9cf5c9965f96c2ade41b9703a985dd96570e4ea174...,0xfc071f946426c4da61b3b024afb26a161840a0ef9007...,19066318
...,...,...,...,...,...,...,...,...,...,...
130,0xae78736Cd615f374D3085123A210448E74Fc6393,0xf9231D28B34CD77A08542f73ca87c4411B1b8B56,6648077906304947453,6.648078,2024-02-19 10:39:47,23,0,0xb829d7de5be10d3792d06d43ea0996278831b606f2e0...,0xa79ac7d85c03c3ac1e2d7f4a68aa4c479b428c015aa3...,19261244
131,0xae78736Cd615f374D3085123A210448E74Fc6393,0x551C5aAdB287D086B47DaBd0cf4d8e83965e79B5,5000000000000000000,5.000000,2024-02-19 11:06:59,183,50,0xbeae086068b4316e3dc9dcbf8fad54a1efd96b5c0d80...,0x7c4d69a1972cf0c64c9597120e60da7da937b9bcb664...,19261378
132,0xae78736Cd615f374D3085123A210448E74Fc6393,0x8fCe57C00d63265755Bbf6af45711f78838596E2,5644595640363383119,5.644596,2024-02-20 05:03:35,188,91,0x540d563d3b7a9acd9e2046b40ea2d43e34bbfd32dffd...,0xb6bb3a67e2a52f02135a0599937efa2cdb4bf6706515...,19266712
133,0xae78736Cd615f374D3085123A210448E74Fc6393,0x684566C9FFcAC7F6A04C3a9997000d2d58C00824,86271338752331706358,86.271339,2024-02-20 09:10:23,318,146,0xf8917edc4adccd598606b8a67dd198369327330f6dc2...,0x8f54a617cb201c36678fb25042a8f87c8f4b32b82b0a...,19267942
